In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

2025/07/30 08:23:03 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/30 08:23:03 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [2]:
client.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/mlops-training/02-experiment_tracking/mlruns/3', creation_time=1753859961092, experiment_id='3', last_update_time=1753859961092, lifecycle_stage='active', name='model-registry-experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops-training/02-experiment_tracking/mlruns/2', creation_time=1753780599458, experiment_id='2', last_update_time=1753780599458, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops-training/02-experiment_tracking/mlruns/0', creation_time=1753780499798, experiment_id='0', last_update_time=1753780499798, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
# client.create_experiment(name="model-registry-experiment")

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(experiment_ids='2', filter_string="",run_view_type=ViewType.ACTIVE_ONLY, max_results=5, order_by=["metrics.rmse ASC"])

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 42184479263240dba64062e77943d5a9, rmse: 6.3604
run id: 7bcff21cc67d4cab99d8aac3eec13d8f, rmse: 6.3828
run id: 4c4cdb679a9946319fbd2635e7a3e601, rmse: 6.5853
run id: cdbb1ce52f9a4720a63d4f97a9eb9c2a, rmse: 11.1673
run id: 558c656fb0c54b239e04fd9b583551e7, rmse: 11.1673


In [7]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [11]:
run_id = "7cd5acfe20ae42dcab48c693f55ea00b"
name = f"runs:/{run_id}/model"

# mlflow.register_model(model_uri=name, name="try-model")

In [8]:
client.search_registered_models()

2025/07/30 08:23:25 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/30 08:23:25 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


[<RegisteredModel: aliases={}, creation_timestamp=1753859636602, deployment_job_id=None, deployment_job_state=None, description='', last_updated_timestamp=1753863518068, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1753859636878, current_stage='None', deployment_job_state=None, description='', last_updated_timestamp=1753859636878, metrics=None, model_id=None, name='try-model', params=None, run_id='', run_link='', source='/home/ubuntu/mlops-training/02-experiment_tracking/mlruns/2/models/m-7defaa2f02234ee88debaeaf77233b3d/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=1>,
  <ModelVersion: aliases=[], creation_timestamp=1753863294380, current_stage='Staging', deployment_job_state=None, description=None, last_updated_timestamp=1753863518068, metrics=None, model_id=None, name='try-model', params=None, run_id='7cd5acfe20ae42dcab48c693f55ea00b', run_link=None, source='models:/m-7defaa2f02234ee88debaeaf77233b3d', status='READY', status_messag

In [9]:
model_name = "try-model"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"{version.version}, stage: {version.current_stage}")

1, stage: None
2, stage: Staging


/tmp/ipykernel_17044/2107327220.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [ ]:
client.transition_model_version_stage(
    name = model_name,
    version=2,
    stage="Staging",
    archive_existing_versions=False
)

/tmp/ipykernel_4038/2459217647.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1753863294380, current_stage='Staging', deployment_job_state=None, description=None, last_updated_timestamp=1753863518068, metrics=None, model_id=None, name='try-model', params=None, run_id='7cd5acfe20ae42dcab48c693f55ea00b', run_link=None, source='models:/m-7defaa2f02234ee88debaeaf77233b3d', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [13]:
# client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')